In [52]:
import numpy as np
import pandas as pd
import pickle
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly.express as px
import plotly.graph_objects as go


### Load dataset

In [66]:
apr = pd.read_csv("../appraisal.csv", on_bad_lines='skip')
usefull_view = apr.groupby(["brand", "model", "version"]).nunique().sort_values("id", ascending=False)
usefull_view = usefull_view.head(300)
useful_names = set(usefull_view.index.map("_".join))
apr["name"] = apr["brand"] + "_" +  apr["model"] + "_" +  apr["version"]
uapr = apr[apr["name"].apply(lambda x: x in useful_names)]
uapr = uapr[uapr["sell_price"] < 1e5]
uapr

,id,brand,model,version,year,km,sell_price,purchase_price,name
0,24,YAMAHA,X-MAX,250,2018.0,40000.0,4990.0,NaN,YAMAHA_X-MAX_250
3,37,YAMAHA,YZF,R1,2005.0,50000.0,4690.0,NaN,YAMAHA_YZF_R1
5,40,KAWASAKI,Z,900,2018.0,22800.0,6590.0,NaN,KAWASAKI_Z_900
7,59,KAWASAKI,VN,900 CLASSIC,2009.0,35000.0,4990.0,NaN,KAWASAKI_VN_900 CLASSIC
17,99,HONDA,[*AJ*] CBR,600 RR,2019.0,10000.0,8190.0,NaN,HONDA_[*AJ*] CBR_600 RR
...,...,...,...,...,...,...,...,...,...
10155,55556,PEUGEOT,TWEET,125,2018.0,10000.0,1590.0,NaN,PEUGEOT_TWEET_125
10157,55582,HONDA,CBF,600 S,2007.0,10000.0,4390.0,NaN,HONDA_CBF_600 S
10158,55591,PIAGGIO,X EVO,125,2011.0,20000.0,1790.0,NaN,PIAGGIO_X EVO_125
10159,55613,YAMAHA,X-MAX,125,2018.0,20000.0,3990.0,NaN,YAMAHA_X-MAX_125


In [73]:
usefull_view

id  year  km  sell_price  \
brand    model      version                                       
KAWASAKI Z          750               118    11  21          27   
SUZUKI   GSR        600               114    13  26          20   
KAWASAKI Z          900               108     6  25          26   
YAMAHA   X-MAX      125               107    19  19          35   
         T-MAX      530               107    14  14          39   
...                                   ...   ...  ..         ...   
HONDA    TRANSALP   XL 700 V            8     5   4           7   
TRIUMPH  BONNEVILLE T 100               8     7   5           7   
KYMCO    K-XCT      300                 8     4   4           5   
         LIKE       125                 8     4   4           7   
YAMAHA   XJ6        600  DIVERSION S    8     6   5           6   

                                      purchase_price  
brand    model      version                           
KAWASAKI Z          750                           10  
SUZUKI   GSR        600                           14  
KAWASAKI Z          900                           10  
YAMAHA   X-MAX      125                           14  
         T-MAX      530                            2  
...                                              ...  
HONDA    TRANSALP   XL 700 V                       1  
TRIUMPH  BONNEVILLE T 100                          2  
KYMCO    K-XCT      300                            2  
         LIKE       125                            2  
YAMAHA   XJ6        600  DIVERSION S               1  

[300 rows x 5 columns]

### Load model 

In [9]:
with open('models/autosk_m1.pkl', 'rb') as f:
    amodel = pickle.load(f)

### Prediction

In [77]:
current_year = 2022

def mk_combo(name):
    return widgets.Combobox(
        options=tuple(uapr[name].unique()),
        description=f"{name}:",
        ensure_option=True,
        value=uapr[name][0]
    )

@interact(
    brand=mk_combo("brand"),
    model=mk_combo("model"),
    version=mk_combo("version"),
    year=widgets.FloatSlider(min=1990, max=2022, step=1, value=2010, description="year:"),
    km=widgets.FloatSlider(min=0, max=80000, step=5000, value=20000, description="km:"),
)
def show(brand, model, version, year, km):
    features = pd.DataFrame({
        "name": pd.Series([f"{brand}_{model}_{version}"]).astype("category"),
        "brand": pd.Series([brand]).astype("category"),
        "model": pd.Series([model]).astype("category"),
        "version": pd.Series([version]).astype("category"),
        "age": current_year - year,
        "km": km,
        "log_km": np.log(km+1),
    })
    
    # print(features)
    
    predicted = amodel.predict(features)[0]
    
    #& (uapr["year"] == current_year - features["age"][0])
    similar = uapr[(uapr["name"] == features["name"][0])]
    # print(similar)
    fig = px.scatter(similar, x="km", y="sell_price", color="year")
    
    fig.add_trace(
        go.Scatter(
            x=[features["km"][0]],
            y=[predicted],
            # symbol=["star"],
            # color=["green"],
            mode="markers",
            marker=dict(
                color="red",
                size=10,
                symbol="star",
            ),
            showlegend=False
        )
    )

    d = fig.to_dict()
    d["data"][0]["type"] = "scatter"

    go.Figure(d).show()
    
# KAWASAKI	Z	750

interactive(children=(Combobox(value='YAMAHA', description='brand:', ensure_option=True, options=('YAMAHA', 'K…